# tool_create_encoded_messages_missing_senders

# Main

In [1]:
VERSION="1.1.19"

In [2]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts
# test

In [3]:
VERSION="1.1.16"
import os
os.environ["HADOOP_USER_NAME"] = "hdfs"

In [4]:
start_year=2021
start_month=1
start_day_in_month=1

end_year=2021
end_month=1
end_day_in_month=5

In [5]:
version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
share_directory =  '/home/jovyan/work/share/'
#share_directory =  '/home/jovyan/share/'

In [6]:
import pandas as pd
from pyspark.sql import functions

In [7]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns

columns = ['CGLOBALMESSAGEID',  'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME']
     

#columns = get_columns_2()
#to count messages sent
#columns = [ 'CSTARTTIME', 'CSENDERENDPOINTID']

In [8]:
sparkSession = dfBasics.getSparkSession()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/15 14:03:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/15 14:03:20 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [9]:
from datetime import datetime, timezone
import pyspark.sql.functions as f

def timestamp(dt):
    return dt.replace(tzinfo=timezone.utc).timestamp() * 1000

startTime = timestamp(datetime(start_year, start_month, start_day_in_month))
endTime = timestamp(datetime(end_year, end_month, end_day_in_month))

print(endTime)

filename_min=str(int(startTime/100000000))

print(filename_min)

# df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').select(columns).show()

1609804800000.0
16094


In [10]:
print("Start: " + str(startTime))
print("End: " + str(endTime))

Start: 1609459200000.0
End: 1609804800000.0


In [11]:
import pyspark.sql.functions as f
import pyspark

#df = sparkSession.read.parquet("hdfs://172.30.17.145:8020/sla_sql_data/*/*").select(columns)
df = sparkSession.read.parquet("hdfs://172.30.17.145:8020/sla_sql_data/" + "*/*").select(columns).filter((f.col('CSTARTTIME') >= startTime) & (f.col('CSTARTTIME') < endTime))
# FIXME reactivate
#.filter((f.col('CSTARTTIME') >= startTime) & (f.col('CSTARTTIME') < endTime)).dropDuplicates()

#df = sparkSession.read.parquet("hdfs://172.30.17.145:8020/sla_sql_data/" + str(15801371) +"*/*").select(columns).filter((f.col('CSTARTTIME') >= startTime) & (f.col('CSTARTTIME') < endTime)).dropDuplicates()


In [12]:
# Find min and max date range
import pyspark.sql.functions as f

# df.select(f.min(f.col('CSTARTTIME')).alias("min"), f.max(f.col('CSTARTTIME')).alias("max")).show()


In [13]:
#senders = sparkSession.read.parquet("/tmp/senders.parquet")
#senders = pd.read_parquet('/tmp/senders' + '.parquet', engine='pyarrow')
#senders = list(senders.toPandas()['CSENDERENDPOINTID'])

In [14]:
#senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').dropDuplicates().toPandas()['CSENDERENDPOINTID'])
#/senders/senders_07.03.2023.parquet
senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/preprocessing/senders/senders_07.03.2023.parquet').dropDuplicates().toPandas()['CSENDERENDPOINTID'])

In [15]:
len(dict.fromkeys(senders))

5697

In [16]:
#senders

In [17]:
#senders_files = sparkSession.read.options(delimiter=',')     .csv('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/senders/senders_files.txt').toPandas()
#senders_files.columns = ['size','filename']    

In [18]:
#senders_files['filename'][0]

In [19]:
'sla_enc_' + senders[0] + '.parquet'

'sla_enc_9df80140-60b6-11e7-ac31-a21bac1b495c.parquet'

In [20]:
#failed = []
#for sender in senders:
#    try:
#        filename = 'sla_enc_' + sender + '.parquet'
#        sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS'); 
#    except Exception as exception: 
#        failed.append(sender)
#        if not 'temporary' in filename:
#            failed.append(filename.split('sla_enc_')[1].split('.')[0])

In [21]:
print('/user/admin/sla/' + version + '/encoded/senders/')

/user/admin/sla/v00001/v00000/encoded/senders/


In [22]:
sc =  sparkSession.sparkContext
URI  = sc._gateway.jvm.java.net.URI
fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(URI("hdfs://172.30.17.145:8020"), sc._jsc.hadoopConfiguration())
failed = []
success = []


for sender in senders:
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        fileExists = fs.exists(sc._jvm.org.apache.hadoop.fs.Path('/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS'))

        if not fileExists:
            failed.append(filename.split('sla_enc_')[1].split('.')[0])
    except Exception as exception: 
            print("Error in file: " + str(exception))


Error in file: can only concatenate str (not "NoneType") to str


In [23]:
#print("senders: " + str(len(senders)) + " failed: " + str(len(dict.fromkeys(failed))))

#len(set(success))

senders: 5697 failed: 2254


0

In [24]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def transform(value):
    return int( _encoder.transform([value])[0])
    
udf_transform = udf(lambda z: transform(z), StringType())

#df2.withColumn("CSENDERENDPOINTID", str( _encoder.transform([df2.CSENDERENDPOINTID])[0])) 
#df2 = df2.withColumn("CSENDERENDPOINTID", udf_transform(df2.CSENDERENDPOINTID)) 

def get_columns(df):
    columns = list(df.limit(1).toPandas().columns)
    columns.remove('CGLOBALMESSAGEID')
    columns.remove('CSLATAT')
    columns.remove('CMESSAGETAT2') 
    columns.remove('CSLADELIVERYTIME')
    columns.remove('CINBOUNDSIZE')
    columns.remove('CSTARTTIME')
    columns.remove('CENDTIME')
    columns.remove('CSLADELIVERYTIME')
    
    return columns

In [25]:
def useSendersOnly(sender=None, dataframe=None):
    #print("Senders size: " + str([sender]))
    return dataframe.where(f.col("CSENDERENDPOINTID").isin(sender)) 

In [26]:
# ## encode columns

from datetime import time
import datetime as dt
import calendar
import pytz
de = pytz.timezone('Europe/Berlin')
from pyspark.sql.types import IntegerType

# long timestamp
def date(x):
    return  dt.datetime.fromtimestamp(float(x) / 1e3, tz=de)

udf_add_year = udf(lambda z: date(z).date().year, IntegerType())
udf_add_month = udf(lambda z: date(z).date().month, IntegerType())
udf_add_day = udf(lambda z: date(z).date().day, IntegerType())
udf_add_hour = udf(lambda z: date(z).time().hour, IntegerType())
udf_add_minute = udf(lambda z: date(z).time().minute, IntegerType())
udf_add_minute = udf(lambda z: date(z).time().minute, IntegerType())


# In[14]:


import numpy as np
import encoder
from pyspark.sql.functions import col


def encode_columns_spark_map(dataframe=None,columns=None, npy='/home/jovyan/work/npy'):
    print(str(columns))
    for column in columns:
        global _encoder
        print (column)
        #_encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
        #_encoder.classes_ = np.load(npy + '/' + column + '.npy')
        
        _encoder = encoders[column]
        
        #dataall[column] = _encoder.transform(dataall[column]) 
        udf_transform = udf(lambda z: transform(z), StringType())
        dataframe=dataframe.withColumn(column, udf_transform(col(column)).cast("Integer"))
        #df3.head()
    return dataframe

def encode_columns_spark(dataframe=None,columns=None, npy='/home/jovyan/work/npy'):
    print(str(columns))
    for column in columns:
        global _encoder
        print (column)
        #_encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
        #_encoder.classes_ = np.load(npy + '/' + column + '.npy')
        
        _encoder = encoders[column]
        
        #dataall[column] = _encoder.transform(dataall[column]) 
        udf_transform = udf(lambda z: transform(z), StringType())
        dataframe=dataframe.withColumn(column, udf_transform(col(column)).cast("Integer"))
        #df3.head()
    return dataframe


# In[15]:


def cast_spark_columns(dataframe=None,columns=[],type="int" ):
    for column in columns:
        dataframe_r = dataframe.withColumn(column, col(column).cast(type))
    return dataframe_r    


# In[16]:



def process(df3):
    df4 = encode_columns_spark(dataframe=df3,columns=columns)
    df5 = df4.withColumn("year", udf_add_year(df4.CSTARTTIME)).withColumn("month", udf_add_month(df4.CSTARTTIME)).withColumn("day", udf_add_day(df4.CSTARTTIME)).withColumn("hour", udf_add_hour(df4.CSTARTTIME)).withColumn("minute", udf_add_minute(df4.CSTARTTIME)) 
    df6=cast_spark_columns(dataframe=df5, columns=['CSTARTTIME', 'CENDTIME','CINBOUNDSIZE','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME'], type='long')
    return df6

# In[17]:

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old


# In[18]:


#import pyspark.sql.functions as f
#ender = senders[0]
#df4 = process(sender=sender,dataframe=df)
#df4.head()
#!mkdir /tmp/enc


# In[19]:





In [27]:
#!mkdir -p /home/jovyan/work/output/enc

In [28]:
if None in senders:
    senders.remove(None)

In [29]:
#None in senders

In [30]:
from os import listdir

#def listdirectory(path=None,filter='.'):
#    return [x for x in listdir(path) if not x.startswith(filter)]    

#_files = listdirectory(path='/home/jovyan/work/output/enc')
#senders = senders[len(_files):]

columns = ['CSTATUS','CSERVICE','CSENDERENDPOINTID','CSENDERPROTOCOL','CRECEIVERPROTOCOL','CRECEIVERENDPOINTID']

In [31]:
npy='/home/jovyan/work/npy'

encoders = {}
for column in columns:
#    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    print(column)
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=False)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    encoders[column] = _encoder

CSTATUS
CSERVICE
CSENDERENDPOINTID
CSENDERPROTOCOL
CRECEIVERPROTOCOL
CRECEIVERENDPOINTID


In [32]:
encoders

{'CSTATUS': TolerantLabelEncoder(),
 'CSERVICE': TolerantLabelEncoder(),
 'CSENDERENDPOINTID': TolerantLabelEncoder(),
 'CSENDERPROTOCOL': TolerantLabelEncoder(),
 'CRECEIVERPROTOCOL': TolerantLabelEncoder(),
 'CRECEIVERENDPOINTID': TolerantLabelEncoder()}

In [33]:
#import pyspark.sql.functions as f
#sender = senders[0]
#print(sender)
#df4 = process(sender=sender,dataframe=df)
#df4.write.mode("overwrite").parquet("/home/jovyan/work/output/enc/sla_enc_" + sender + ".parquet")

In [34]:
#sender = senders[0]
#for sender in failed:
#    dfOfSender = useSendersOnly(sender=sender,dataframe=df)

#    dataCount = dfOfSender.count()
#    biggerCounter = 0
    
#    if dataCount < 1 :
#        print("No data for sender: " + sender + " count: " + str(dataCount) + " Bigger count: " + str(biggerCounter))
#    else:
#        biggerCounter = biggerCounter + 1
        
    
#print("Process sender: " + sender + " Data count: " + str(dfOfSender.count()))
#df10 = process(dfOfSender)
#df10.toPandas()

In [38]:
import pyspark.sql.functions as f
import pyspark

dfOfSender = df.where(f.col("CSENDERENDPOINTID").isin(failed)).persist(pyspark.StorageLevel.MEMORY_ONLY) 
#dfOfSender.explain()
#print("Process sender: " + sender + " Data count: " + str(dfOfSender.count()))
df10 = process(df)#.repartition(1)
#df10.explain()
df10.write.mode("append").parquet("hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/" + VERSION + "/" + str(start_year) + "/" + str(start_day_in_month) + "." + str(start_month) + "."  + str(start_year)  + "-" +  str(end_day_in_month) + "." + str(end_month) + "."  + str(end_year) + "/sla_enc_" + sender + '.parquet') 
#df10.write.mode("overwrite").parquet("/home/jovyan/work/output/enc/test/" + str(start_year) + "/sla_enc_" + sender + "-" + str(start_year) + "-" + str(start_month) + "-" + str(start_day_in_month) + ".parquet")

23/03/15 14:49:20 WARN CacheManager: Asked to cache already cached data.


Process sender: 3fe7dc81-519c-11eb-b000-d76cac1b495c Data count: 741606


In [ ]:
import pyspark.sql.functions as f


#sender = senders[0]
#for sender in failed:
#    dfOfSender = useSendersOnly(sender=sender,dataframe=df)
#    #print("Process sender: " + sender + " Data count: " + str(dfOfSender.count()))
#    df10 = process(dfOfSender).repartition(1)
#    df10.write.mode("append").parquet("hdfs://172.30.17.145:8020/preprocessing/enc/missing_senders/" + VERSION + "/" + str(start_year) + "/" + str(start_day_in_month) + "." + str(start_month) + "."  + str(start_year)  + "-" +  str(end_day_in_month) + "." + str(end_month) + "."  + str(end_year) + "/sla_enc_" + sender + '.parquet') 
#    #df10.write.mode("overwrite").parquet("/home/jovyan/work/output/enc/test/" + str(start_year) + "/sla_enc_" + sender + "-" + str(start_year) + "-" + str(start_month) + "-" + str(start_day_in_month) + ".parquet")

In [ ]:
#  papermill /home/jovyan/work/tool_create_encoded_messages_missing_senders.ipynb  /home/jovyan/work/integration/papermill-jobs/job-output/out_tool_create_encoded_messages_missing_senders.ipynb -p start_year 2023  -p start_month 1 -p start_day_in_month 1 -p end_year 2023 -p end_month 2  -p end_day_in_month 1